In [1]:
!pip install vtk
!pip install plotly

In [2]:
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
import vtk


# Load the provided 3D dataset

In [3]:
reader = vtk.vtkXMLImageDataReader()
reader.SetFileName('mixture.vti')
reader.Update()

# Extract data from the VTI file

In [4]:
image_data = reader.GetOutput()
dimensions = image_data.GetDimensions()
spacing = image_data.GetSpacing()
origin = image_data.GetOrigin()

# Convert VTK data to NumPy array

In [5]:
volume = np.array(image_data.GetPointData().GetScalars()).reshape(dimensions, order='F')

# Define initial isovalue

In [6]:
initial_isovalue = 0.0

# Create Isosurface plot

In [7]:
x=np.linspace(origin[0], origin[0] + spacing[0] * (dimensions[0]-1), dimensions[0]),
y=np.linspace(origin[1], origin[1] + spacing[1] * (dimensions[1]-1), dimensions[1]),
z=np.linspace(origin[2], origin[2] + spacing[2] * (dimensions[2]-1), dimensions[2]),
X, Y, Z = np.meshgrid(x, y, z, indexing='ij')
fig_isosurface = go.FigureWidget()
isosurface = fig_isosurface.add_trace(go.Isosurface(
    x = X.flatten(),
    y = Y.flatten(),
    z = Z.flatten(),
    showscale=False,
    value=volume.flatten(), 
    isomin=initial_isovalue,
    isomax=initial_isovalue,
    cmin=np.min(volume),
    cmax=np.max(volume),
    colorscale='plasma'
))
fig_isosurface.update_layout(
    scene=dict(
        xaxis=dict(
            title='x',
            tickvals=[],
            showticklabels=False
        ),
        yaxis=dict(
            title='y',
            tickvals=[],
            showticklabels=False
        ),
        zaxis=dict(
            title='z',
            tickvals=[],
            showticklabels=False
        )
    )
)

FigureWidget({
    'data': [{'cmax': 0.43280163,
              'cmin': -0.99355406,
              'colorscale': [[0.0, '#0d0887'], [0.1111111111111111, '#46039f'],
                             [0.2222222222222222, '#7201a8'], [0.3333333333333333,
                             '#9c179e'], [0.4444444444444444, '#bd3786'],
                             [0.5555555555555556, '#d8576b'], [0.6666666666666666,
                             '#ed7953'], [0.7777777777777778, '#fb9f3a'],
                             [0.8888888888888888, '#fdca26'], [1.0, '#f0f921']],
              'isomax': 0.0,
              'isomin': 0.0,
              'showscale': False,
              'type': 'isosurface',
              'uid': 'c9c971f8-9d66-4255-a203-648aebf03ee1',
              'value': array([-0.04087147, -0.02041778, -0.0047331 , ..., -0.9426848 , -0.8584828 ,
                              -0.72851294], dtype=float32),
              'x': array([7.45000000e-05, 7.45000000e-05, 7.45000000e-05, ..., 1.48999926e+0

# Create Histogram

In [8]:
fig_histogram = go.FigureWidget()
histogram_data =volume.flatten()
histogram = fig_histogram.add_trace(go.Histogram(
    x=histogram_data,
    marker=dict(color='blue'),
    opacity=0.7,
    nbinsx=30,
    name='Histogram'
))
fig_histogram.update_layout(
    xaxis_title_text='Vortex scaler values',
    yaxis_title_text='Frequency'
)

FigureWidget({
    'data': [{'marker': {'color': 'blue'},
              'name': 'Histogram',
              'nbinsx': 30,
              'opacity': 0.7,
              'type': 'histogram',
              'uid': 'a4ca1aee-d45c-43e8-8d2d-64b5bb333ad7',
              'x': array([-0.04087147, -0.02041778, -0.0047331 , ..., -0.9426848 , -0.8584828 ,
                          -0.72851294], dtype=float32)}],
    'layout': {'template': '...',
               'xaxis': {'title': {'text': 'Vortex scaler values'}},
               'yaxis': {'title': {'text': 'Frequency'}}}
})

# Creating slider and buttons and merging all components together

In [9]:
def func(change):
    isosurface.data[0].isomin = change.new
    isosurface.data[0].isomax = change.new
    isosurface.update_traces()
    histogram_data = volume[(volume >= change.new - 0.25) & (volume <= change.new + 0.25)].flatten()
    fig_histogram.data[0].x = histogram_data

def button_click(reset_button):
    isosurface.data[0].isomin = 0.00
    isosurface.data[0].isomax = 0.00
    slider.value=0.00
    isosurface.update_traces()
    histogram_data = volume.flatten()
    fig_histogram.data[0].x = histogram_data
    
slider = widgets.FloatSlider(value=initial_isovalue, min=np.min(volume), max=np.max(volume), step=0.01, description="isoval:",continuous_update=False,layout=widgets.Layout(width='50%'))

# gui = widgets.interactive_output(func, {'isovalue':slider})
slider.observe(func, 'value')
reset_button = widgets.Button(description='Reset')
reset_button.on_click(button_click)
x = widgets.VBox([widgets.HBox([slider,reset_button]), widgets.HBox([fig_isosurface,fig_histogram])])
button_click(None)
display(x)